In [1]:
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
import time

In [3]:
from election_campaign import general_trends

In [4]:
alltweets = general_trends.get_alltweets(data_type='pandas.dataframe')

In [5]:
alltweets.head(10)

,time_created,tweet,loca_tion
0,2022-10-23T23:59:52+00:00,@PeterObi @DavidIbiyeomie I want set up a farm...,"Lagos, Nigeria"
1,2022-10-23T23:59:43+00:00,"Wait, I don't understand. How on earth will th...","Lagos, Nigeria"
2,2022-10-23T23:59:38+00:00,I won’t rule Nigeria from Dubai as part-time p...,Nigeria
3,2022-10-23T23:59:36+00:00,"Kanu: Ohanaeze, Igbo leaders to meet Buhari, i...",Nigeria
4,2022-10-23T23:59:23+00:00,@JckMent @channelstv Are you a novice on Twitt...,Lagos
5,2022-10-23T23:59:19+00:00,@hz_dikko @d_kenzo_ @PeterObi @DavidIbiyeomie ...,"Ibadan, Nigeria🇳🇬"
6,2022-10-23T23:59:16+00:00,The way the Dpo killed that suspect in public ...,"Lagos, Nigeria"
7,2022-10-23T23:59:14+00:00,"PDP’ll win Anambra, says Okowa - https://t.co/...",Nigeria
8,2022-10-23T23:59:11+00:00,@PeterObi @DavidIbiyeomie Happy birthday sir,Nigeria
9,2022-10-23T23:59:11+00:00,24-10-2022\n@PeterObi wished pastor @DavidIbiy...,Nigeria


In [6]:
alltweets.to_csv('citizensvoice_dataset.csv')